In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()
_base_ = '../../data/2013_vast_challenge/mc3_netflow/nf/'
df_orig = pl.concat([pl.read_csv(_base_ + 'nf-chunk1.csv'), pl.read_csv(_base_ + 'nf-chunk2.csv'), pl.read_csv(_base_ + 'nf-chunk3.csv')])
# df_orig = pl.concat([pl.read_csv(_base_ + 'nf-chunk2.csv')])
df_orig = df_orig.rename({'TimeSeconds':'secs',                  'parsedDate':'timestamp',                'dateTimeStr':'timestamp_str',
                          'ipLayerProtocol':'pro_str',           'ipLayerProtocolCode':'pro',             'firstSeenSrcIp':'sip',
                          'firstSeenDestIp':'dip',               'firstSeenSrcPort':'spt',                'firstSeenDestPort':'dpt',
                          'moreFragments':'mfrag',               'contFragments':'cfrag',                 'durationSeconds':'dur',
                          'firstSeenSrcPayloadBytes':'soct_pay', 'firstSeenDestPayloadBytes':'doct_pay',  'firstSeenSrcTotalBytes':'soct',
                          'firstSeenDestTotalBytes':'doct',      'firstSeenSrcPacketCount':'spkt',        'firstSeenDestPacketCount':'dpkt',
                          'recordForceOut':'out'})
df_orig = rt.columnsAreTimestamps(df_orig, 'timestamp')
df       = df_orig.sample(500_000)
_cd_ = rt.chordDiagram(df, [('sip','dip')], 
                       link_style='bundled', color_by=rt.createTField('dip', 'ipv4_cidr_16'), link_color='vary', link_opacity=0.2, beta=0.85,
                       skeleton_algorithm='kmeans', track_routes=True, w=512, h=512) # , equal_size_nodes=True)
_cd_._repr_svg_()
#rt.tile([_cd_, _cd_.skeleton_svg], spacer=20)

In [ ]:
_cd_.fmto_entry, _cd_.fmto_exit, _cd_.fmto_fm_pos, _cd_.fmto_to_pos

In [ ]:
#
# Maps out the first layer from the node to the skeleton networks entrance
#
svg = [f'<svg width="{_cd_.w}" height="{_cd_.h}" x="0" y="0" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect x="0" y="0" width="{_cd_.w}" height="{_cd_.h}" fill="#ffffff" stroke="#000000" />')
for _xy_ in _cd_.fmto_entry:
    _xy2_ = _cd_.fmto_entry[_xy_]
    svg.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{_xy2_[0]}" y2="{_xy2_[1]}" stroke="#000000" stroke-width="0.2" />')    
svg.append('</svg>')

svg2 = [f'<svg width="{_cd_.w}" height="{_cd_.h}" x="0" y="0" xmlns="http://www.w3.org/2000/svg">']
svg2.append(f'<rect x="0" y="0" width="{_cd_.w}" height="{_cd_.h}" fill="#ffffff" stroke="#000000" />')
for _xy_ in _cd_.fmto_exit:
    _xy2_ = _cd_.fmto_exit[_xy_]
    svg2.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{_xy2_[0]}" y2="{_xy2_[1]}" stroke="#000000" stroke-width="0.2" />')    
svg2.append('</svg>')

rt.tile([''.join(svg), ''.join(svg2)], spacer=20)